<a href="https://colab.research.google.com/github/PharahMain/Flight-Delay-Predictor/blob/master/flightstats_client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

In [3]:
# Install spark-related depdencies for Python
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 215.7MB 56kB/s 
     |████████████████████████████████| 204kB 52.3MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=0332e7539e5ece4a3a48caad24865c37f6a0895277b9d159ff38c5c3e94d25a6
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [5]:
# Point Colaboratory to Google Drive

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.sql.functions import isnan, when, count, col

from datetime import datetime, date, timedelta
import time

In [0]:
ARR_JSON_PATH = f"/content/gdrive/My Drive/Colab Datasets/flights_data/arrivals/{date.today()}.json"
DEP_JSON_PATH = f"/content/gdrive/My Drive/Colab Datasets/flights_data/departures/{date.today()}.json"
WEATH_JSON_PATH = f"/content/gdrive/My Drive/Colab Datasets/flights_data/weather/{date.today()}.json"
APP_NAME = 'Flight Delay Predictor'
SPARK_URL = 'local[*]'
RANDOM_SEED = 263646
TRAINING_DATA_RATIO = 0.7

In [0]:
spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()

In [21]:
df_arr = spark.read.json(ARR_JSON_PATH)
df_arr.show(5)

+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+----------------------+--------------------+---------------------+--------------------+------------------+----------+------------+--------------------+--------------------+------+
|    airportResources|arrivalAirportFsCode|         arrivalDate|carrierFsCode|          codeshares|              delays|departureAirportFsCode|       departureDate|divertedAirportFsCode|     flightDurations|   flightEquipment|  flightId|flightNumber|    operationalTimes|            schedule|status|
+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+----------------------+--------------------+---------------------+--------------------+------------------+----------+------------+--------------------+--------------------+------+
|      [, TBIT,, C9,]|                 LAX|[2020-01-17T13:10...|           QR|     [[3515, UL, L]]| 

Let's first drop all the unnecessary columns.

In [26]:
df_arr = df_arr.drop('airportResources', 'codeshares', 'departureDate',
            'divertedAirportFsCode', 'operationalTimes', 'schedule', 'status')
df_arr.show(5)

+--------------------+--------------------+-------------+--------------------+----------------------+--------------------+------------------+----------+------------+
|arrivalAirportFsCode|         arrivalDate|carrierFsCode|              delays|departureAirportFsCode|     flightDurations|   flightEquipment|  flightId|flightNumber|
+--------------------+--------------------+-------------+--------------------+----------------------+--------------------+------------------+----------+------------+
|                 LAX|[2020-01-17T13:10...|           QR|      [, 13, 52, 60]|                   DOH|[, 1021, 985,, 10...|[77L, 77L, A7-BBA]|1028199581|         739|
|                 LAX|[2020-01-17T13:15...|           QF|[205, 247, 226, 240]|                   SYD|[, 763, 830, 57, ...|[388, 388, VH-OQJ]|1028197993|          17|
|                 LAX|[2020-01-17T13:50...|           SU|          [14, 23,,]|                   SVO|[, 745, 770, 15, ...|[332, 332, VQ-BBE]|1028204787|         106|
|   

In [24]:
df_arr.select(1)

TypeError: ignored

In [23]:
df_weath = spark.read.json(WEATH_JSON_PATH)
df_weath.show(5)

+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+
|apparentTemperature|cloudCover|dewPoint|humidity|               icon|ozone|precipIntensity|precipProbability|precipType|pressure|            summary|temperature|      time|uvIndex|visibility|windBearing|windGust|windSpeed|
+-------------------+----------+--------+--------+-------------------+-----+---------------+-----------------+----------+--------+-------------------+-----------+----------+-------+----------+-----------+--------+---------+
|              53.33|      0.93|   48.75|    0.84|               rain|306.5|         0.0474|             0.87|      rain|  1020.0|         Light Rain|      53.33|1579248000|      0|      4.94|        118|    9.85|     7.89|
|               53.2|      0.88|   49.85|    0.88|               rain|303.5|         0.0262|            